In [1]:
'''
from sklearn.metrics import roc_auc_score
import numpy as np

# Load validation predictions
xgb_val_preds = np.load('/kaggle/input/01-xgboost/01_xgb_val_preds.npy')
dnn_val_preds = np.load('/kaggle/input/02-dnn/dnn_tuned_val_preds.npy')

# Load validation labels
y_val = np.load('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/val/labels/labels.npy')

# Try different ensemble weights
weights = np.arange(0, 1.05, 0.05)

best_auc = 0
best_weight = None

print("📊 AUC scores for different weights (XGB:DNN):")
for w in weights:
    blended = w * xgb_val_preds + (1 - w) * dnn_val_preds
    auc = roc_auc_score(y_val, blended)
    print(f"  {w:.2f} : {auc:.5f}")
    
    if auc > best_auc:
        best_auc = auc
        best_weight = w

print(f"\n✅ Best weight for XGBoost: {best_weight:.2f} (DNN: {1 - best_weight:.2f})")
print(f"🏆 Best Validation AUC: {best_auc:.5f}")
'''

'\nfrom sklearn.metrics import roc_auc_score\nimport numpy as np\n\n# Load validation predictions\nxgb_val_preds = np.load(\'/kaggle/input/01-xgboost/01_xgb_val_preds.npy\')\ndnn_val_preds = np.load(\'/kaggle/input/02-dnn/dnn_tuned_val_preds.npy\')\n\n# Load validation labels\ny_val = np.load(\'/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/val/labels/labels.npy\')\n\n# Try different ensemble weights\nweights = np.arange(0, 1.05, 0.05)\n\nbest_auc = 0\nbest_weight = None\n\nprint("📊 AUC scores for different weights (XGB:DNN):")\nfor w in weights:\n    blended = w * xgb_val_preds + (1 - w) * dnn_val_preds\n    auc = roc_auc_score(y_val, blended)\n    print(f"  {w:.2f} : {auc:.5f}")\n    \n    if auc > best_auc:\n        best_auc = auc\n        best_weight = w\n\nprint(f"\n✅ Best weight for XGBoost: {best_weight:.2f} (DNN: {1 - best_weight:.2f})")\nprint(f"🏆 Best Validation AUC: {best_auc:.5f}")\n'

In [2]:
'''
import pandas as pd

# Load test predictions
xgb_test_preds = np.load('/kaggle/input/01-xgboost/01_xgb_test_preds.npy')
dnn_test_preds = np.load('/kaggle/input/02-dnn/02_dnn_tuned_test_preds.npy')
test_ids = np.load('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/test/ids/ids.npy')

# Weighted average using best weights
best_weight = 0.45  # example — replace with the best one found
ensemble_test_preds = best_weight * xgb_test_preds + (1 - best_weight) * dnn_test_preds

# Save submission file
submission = pd.DataFrame({
    'id': test_ids,
    'label': ensemble_test_preds
})
submission.to_csv('/kaggle/working/submission.csv', index=False)

print("🚀 Final ensemble submission file saved: submission.csv")
'''

'\nimport pandas as pd\n\n# Load test predictions\nxgb_test_preds = np.load(\'/kaggle/input/01-xgboost/01_xgb_test_preds.npy\')\ndnn_test_preds = np.load(\'/kaggle/input/02-dnn/02_dnn_tuned_test_preds.npy\')\ntest_ids = np.load(\'/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/test/ids/ids.npy\')\n\n# Weighted average using best weights\nbest_weight = 0.45  # example — replace with the best one found\nensemble_test_preds = best_weight * xgb_test_preds + (1 - best_weight) * dnn_test_preds\n\n# Save submission file\nsubmission = pd.DataFrame({\n    \'id\': test_ids,\n    \'label\': ensemble_test_preds\n})\nsubmission.to_csv(\'/kaggle/working/submission.csv\', index=False)\n\nprint("🚀 Final ensemble submission file saved: submission.csv")\n'

In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score

# Load validation predictions
xgb_val = np.load('/kaggle/input/01-xgboost/01_xgb_val_preds.npy')
dnn_val = np.load('/kaggle/input/02-dnn/dnn_tuned_val_preds.npy')
lr_val = np.load('/kaggle/input/00-logistic-regression/00_lr_val_preds.npy')

# Load ground truth
y_val = np.load('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/val/labels/labels.npy')


In [4]:
best_auc = 0
best_weights = None

print("📊 AUC scores for XGB + DNN + LR combinations:\n")

# Try weights in 10% steps (must sum to 1)
for xgb_w in np.arange(0, 1.1, 0.1):
    for dnn_w in np.arange(0, 1.1 - xgb_w, 0.1):
        lr_w = 1.0 - xgb_w - dnn_w
        if lr_w < 0: continue
        
        ensemble_val = xgb_w * xgb_val + dnn_w * dnn_val + lr_w * lr_val
        auc = roc_auc_score(y_val, ensemble_val)
        print(f"XGB: {xgb_w:.1f}, DNN: {dnn_w:.1f}, LR: {lr_w:.1f}  -> AUC: {auc:.5f}")
        
        if auc > best_auc:
            best_auc = auc
            best_weights = (xgb_w, dnn_w, lr_w)

print(f"\n✅ Best Weights → XGB: {best_weights[0]:.2f}, DNN: {best_weights[1]:.2f}, LR: {best_weights[2]:.2f}")
print(f"🏆 Best Validation AUC: {best_auc:.5f}")


📊 AUC scores for XGB + DNN + LR combinations:

XGB: 0.0, DNN: 0.0, LR: 1.0  -> AUC: 0.94440
XGB: 0.0, DNN: 0.1, LR: 0.9  -> AUC: 0.94504
XGB: 0.0, DNN: 0.2, LR: 0.8  -> AUC: 0.94552
XGB: 0.0, DNN: 0.3, LR: 0.7  -> AUC: 0.94605
XGB: 0.0, DNN: 0.4, LR: 0.6  -> AUC: 0.94621
XGB: 0.0, DNN: 0.5, LR: 0.5  -> AUC: 0.94637
XGB: 0.0, DNN: 0.6, LR: 0.4  -> AUC: 0.94639
XGB: 0.0, DNN: 0.7, LR: 0.3  -> AUC: 0.94676
XGB: 0.0, DNN: 0.8, LR: 0.2  -> AUC: 0.94692
XGB: 0.0, DNN: 0.9, LR: 0.1  -> AUC: 0.94669
XGB: 0.0, DNN: 1.0, LR: 0.0  -> AUC: 0.94637
XGB: 0.1, DNN: 0.0, LR: 0.9  -> AUC: 0.94481
XGB: 0.1, DNN: 0.1, LR: 0.8  -> AUC: 0.94518
XGB: 0.1, DNN: 0.2, LR: 0.7  -> AUC: 0.94580
XGB: 0.1, DNN: 0.3, LR: 0.6  -> AUC: 0.94619
XGB: 0.1, DNN: 0.4, LR: 0.5  -> AUC: 0.94639
XGB: 0.1, DNN: 0.5, LR: 0.4  -> AUC: 0.94657
XGB: 0.1, DNN: 0.6, LR: 0.3  -> AUC: 0.94657
XGB: 0.1, DNN: 0.7, LR: 0.2  -> AUC: 0.94669
XGB: 0.1, DNN: 0.8, LR: 0.1  -> AUC: 0.94616
XGB: 0.1, DNN: 0.9, LR: 0.0  -> AUC: 0.94609
XGB: 0.2

In [5]:
# Load test predictions
xgb_test = np.load('/kaggle/input/01-xgboost/01_xgb_test_preds.npy')
dnn_test = np.load('/kaggle/input/02-dnn/02_dnn_tuned_test_preds.npy')
lr_test = np.load('/kaggle/input/00-logistic-regression/00_lr_test_preds.npy')
test_ids = np.load('/kaggle/input/qcd-tt-jet-tagging-hsf-india-bangalore/test/ids/ids.npy')

# Use best weights from earlier
xgb_w, dnn_w, lr_w = best_weights
ensemble_test = xgb_w * xgb_test + dnn_w * dnn_test + lr_w * lr_test

# Save submission
import pandas as pd
submission = pd.DataFrame({
    'id': test_ids,
    'label': ensemble_test
})
submission.to_csv('/kaggle/working/submission.csv', index=False)

print("🚀 Final LR+DNN+XGB ensemble submission saved: submission.csv")


🚀 Final LR+DNN+XGB ensemble submission saved: submission.csv
